# NExT-GQA — BLIP-2 Feature Extraction
Chạy notebook này trên **Google Colab** với GPU (T4 hoặc A100).

**Checklist trước khi chạy:**
- [ ] Runtime → Change runtime type → **GPU** (T4 hoặc A100)
- [ ] Google Drive đã có thư mục `vidor_videos/` (cấu trúc VidOR gốc)

## 1. Kiểm tra GPU

In [ ]:
!nvidia-smi

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Clone repo

In [ ]:
import os

REPO_URL = 'https://github.com/YOUR_USERNAME/TFVTG.git'  # <-- đổi lại
REPO_DIR = '/content/TFVTG'

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print('Repo already cloned, pulling latest...')
    !git -C {REPO_DIR} pull

%cd {REPO_DIR}
!ls dataset/nextgqa/

## 4. Cấu hình đường dẫn
Chỉ cần chỉnh **2 biến** bên dưới.

In [ ]:
# Thư mục VidOR videos trên Drive
# Cấu trúc bên trong: {folder}/{video_id}.mp4
# Ví dụ: .../nextqa/videos/0034/4740345442.mp4
DRIVE_VIDOR_ROOT = '/content/drive/MyDrive/KLTN/VideoMind-Dataset/nextqa/videos'  # <-- đổi nếu cần

# Thư mục lưu .npy features trên Drive (để không mất khi session reset)
DRIVE_FEATURES_OUT = '/content/drive/MyDrive/KLTN/nextgqa_blip2_features'  # <-- đổi nếu cần

print('VIDOR_ROOT   :', DRIVE_VIDOR_ROOT)
print('FEATURES_OUT :', DRIVE_FEATURES_OUT)

## 5. Symlink videos + features vào repo
Annotation files đã có trong repo (đã push), chỉ cần symlink phần nặng.

In [ ]:
import os

# Tạo thư mục output trên Drive nếu chưa có
os.makedirs(DRIVE_FEATURES_OUT, exist_ok=True)

nextgqa_dir = f'{REPO_DIR}/dataset/nextgqa'

# Symlink videos (VidOR) → không copy, quá nặng
videos_link = f'{nextgqa_dir}/videos'
if not os.path.exists(videos_link):
    os.symlink(DRIVE_VIDOR_ROOT, videos_link)
    print(f'Symlinked videos: {videos_link} -> {DRIVE_VIDOR_ROOT}')
else:
    print(f'videos symlink already exists')

# Symlink blip2_features → lưu thẳng vào Drive qua symlink
features_link = f'{nextgqa_dir}/blip2_features'
if not os.path.exists(features_link):
    os.symlink(DRIVE_FEATURES_OUT, features_link)
    print(f'Symlinked features: {features_link} -> {DRIVE_FEATURES_OUT}')
else:
    print(f'blip2_features symlink already exists')

print('\ndataset/nextgqa/ contents:')
!ls -la {nextgqa_dir}

## 6. Sanity check — tìm thử vài videos

In [ ]:
import json, csv, os

nextgqa_dir = f'{REPO_DIR}/dataset/nextgqa'

with open(f'{nextgqa_dir}/map_vid_vidorID.json') as f:
    mapping = json.load(f)

test_vids = set()
with open(f'{nextgqa_dir}/test.csv') as f:
    for row in csv.DictReader(f):
        test_vids.add(row['video_id'])

print(f'Test videos: {len(test_vids)}')
print('\nChecking first 5 video paths:')
found = 0
for vid in list(test_vids)[:5]:
    rel = mapping.get(vid)
    if not rel:
        print(f'  {vid}: NO MAPPING')
        continue
    folder, vid_id = rel.split('/')
    found_path = None
    for ext in ['.mp4', '.avi', '.mkv']:
        # Flat: {folder}/{video_id}.ext  (most common)
        p = f'{DRIVE_VIDOR_ROOT}/{folder}/{vid_id}{ext}'
        if os.path.exists(p):
            found_path = p
            break
        # Nested: {folder}/{video_id}/{video_id}.ext  (VidOR original)
        p2 = f'{DRIVE_VIDOR_ROOT}/{folder}/{vid_id}/{vid_id}{ext}'
        if os.path.exists(p2):
            found_path = p2
            break
    if found_path:
        print(f'  {vid} -> FOUND: {found_path}')
        found += 1
    else:
        print(f'  {vid} -> NOT FOUND (checked {DRIVE_VIDOR_ROOT}/{folder}/{vid_id}[.mp4|/...])')

print(f'\nFound {found}/5 — nếu 0/5 kiểm tra lại DRIVE_VIDOR_ROOT')

## 7. Install dependencies
> ⚠️ `salesforce-lavis` mất ~5-10 phút, chỉ cần chạy 1 lần per session.

In [ ]:
# 1. Pin tokenizers (pre-built wheel, tránh build Rust)
!pip install "tokenizers==0.15.2" -q --prefer-binary

# 2. Cài salesforce-lavis (BLIP-2)
!pip install salesforce-lavis -q --prefer-binary

# 3. Fix numpy 2.x vs opencv ABI conflict:
#    salesforce-lavis cài opencv 4.5.x (compile với numpy 1.x) → crash với numpy 2.x
#    Upgrade opencv lên version hỗ trợ numpy 2.x
!pip install "opencv-python-headless>=4.9.0" -q --prefer-binary

# 4. Cài decord
!pip install decord tqdm -q

print("Done! Verifying cv2 import...")
import cv2
print(f"  cv2 version: {cv2.__version__} ✓")

## 8. Chạy Feature Extraction

- Extract cả **val + test** (1557 videos tổng)
- Tự **resume** nếu session bị ngắt — chạy lại cell này là đủ
- Features lưu thẳng vào Drive qua symlink → an toàn

In [ ]:
!python feature_extraction_nextgqa.py \
    --vidor_root {DRIVE_VIDOR_ROOT} \
    --save_root  dataset/nextgqa/blip2_features \
    --splits test \
    --fps 3 \
    --batch_size 64

## 9. Kiểm tra kết quả

In [ ]:
import numpy as np, os

npy_files = [f for f in os.listdir(DRIVE_FEATURES_OUT) if f.endswith('.npy')]
print(f'Extracted: {len(npy_files)} / 1557 videos')

if npy_files:
    sample = np.load(os.path.join(DRIVE_FEATURES_OUT, npy_files[0]))
    print(f'Sample shape : {sample.shape}  (expected [T, 32, 256])')
    print(f'dtype        : {sample.dtype}   (expected float16)')

---
## Bước 6: Grounder — TFVTG temporal proposals

Sau khi extract features xong, chạy Grounder để sinh top-5 temporal proposals cho mỗi câu hỏi.
- Input : `llm_outputs_test.json` (query rewrite từ Bước 5) + BLIP-2 features
- Output: `grounder_outputs_test.json` (~5553 entries, mỗi entry có `top5_proposals`)
- Tự **resume** nếu bị ngắt
- Cần GPU (BLIP-2)

In [ ]:
# Dry run trước: 10 questions để kiểm tra output format
!python -m nextgqa.grounder --split test --dry_run

In [ ]:
# Chạy full (5553 questions) — tự resume nếu ngắt
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True \
python -m nextgqa.grounder --split test

In [ ]:
# Kiểm tra kết quả Grounder
import json

with open('/content/TFVTG/dataset/nextgqa/grounder_outputs_test.json') as f:
    results = json.load(f)

print(f'Total entries: {len(results)} / 5553')

# Show sample
r = results[0]
print(f'\nSample:')
print(f'  video_id  : {r["video_id"]}')
print(f'  qid       : {r["qid"]}')
print(f'  question  : {r["question"][:60]}...')
print(f'  type      : {r["type"]}')
print(f'  duration  : {r["duration"]}s')
print(f'  gt_segments: {r["gt_segments"]}')
print(f'  top5_proposals:')
for i, p in enumerate(r['top5_proposals']):
    print(f'    #{i+1}: [{p[0]:.1f}s, {p[1]:.1f}s] conf={p[2]:.3f}')